In [1]:
%%capture
import numpy as np
from functools import reduce
import time
%run commute.ipynb
%run diagonalize.ipynb
%run phase.ipynb
%run QDrift.ipynb
%run hamiltonian.ipynb

In [2]:
def Simulation_cost(Chs, CPs, custom=False):
    """ Returns the cost of the simulation,
    total costs, individual costs in [crzs, toffolis]
    """
    crzs, Toffolis = 0, 0
    icosts = []
    n = len(CPs[0][0])
    for Ch, CP in zip(Chs, CPs):
        _, CZ, _ = diag_results(CP, True)
        _, _, cost = logic_min(CZ, Ch, custom)
        crzs += cost[0]
        Toffolis += cost[1]
        icosts.append([cost[0], cost[1]])
    tcost = [crzs, Toffolis]
    return np.array(tcost), np.array(icosts)

def Hs_sum(CHs, Chs):
    """ Regroups the cliques according to coefficients
    """
    tolerance = 1e-3
    SHs, Shs = [], []
    for CH, Ch in zip(CHs, Chs):
        SH, Sh = [], []
        idx = np.argsort(Ch)
        sCH, sCh = np.array(CH)[idx][::-1], np.array(Ch)[idx][::-1]
        while len(sCh) > 0:
            if np.abs(sCh[0]) < tolerance:
                break
            Sh.append(sCh[0])
            SH.append(np.sum(sCH, axis=0))
            sCh = sCh - sCh[0]
            cut = (-1*tolerance > sCh) | (sCh > tolerance)
            sCh = sCh[cut]
            sCH = sCH[cut]
        SHs.append(SH); Shs.append(Sh) 
    return SHs, Shs

def Hs_sum_costs(SHs, Shs, icosts):
    Hs_s, hs_s, costs = [], [], []
    for SH, Sh, icost in zip(SHs, Shs, icosts):
        for H, h in zip(SH, Sh):
            Hs_s.append(H); hs_s.append(h), costs.append(icost)
    return np.array(Hs_s), np.array(hs_s), costs



In [3]:
n = 6
Jx, Jy, Jz, h = np.random.normal(loc=0, scale=1, size=4)
Jx, Jy, Jz, h = -1.0614106051873797, -0.7142265195762633, -1.27989762445829, -0.6121276477817922
print('Coefficients: ', Jx, Jy, Jz, h)
# G = [[0,1], [0,5], [1,2], [1,4], [2,3], [3,4], [3,8], [4,5], [4,7], [5,6], [6,7], [7,8]]
# print('Graph: ', G)
Hm, CHs, Chs, CPs = Heisenberg_1d(n, Jx, Jy, Jz, h, True)
Hm, Hs, hs, Ps = Heisenberg_1d(n, Jx, Jy, Jz, h, False)
tcost, icosts = Simulation_cost(Chs, CPs, True)
SHs, Shs = Hs_sum(CHs, Chs)
Hs_s, hs_s, icosts = Hs_sum_costs(SHs, Shs, icosts)
print(CPs)

Coefficients:  -1.0614106051873797 -0.7142265195762633 -1.27989762445829 -0.6121276477817922
crzs, toffolis, thetas:
1 3 [3.098953491617447, 1.0614106051873797]
crzs, toffolis, thetas:
1 3 [2.1426795587287897, 0.7142265195762629]
crzs, toffolis, thetas:
1 3 [2.4434924338047166, 1.2798976244582896]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
crzs, toffolis, thetas:
1 0 [0.3060638238908959]
[['XXIIII', 'IXXIII', 'IIXXII', 'IIIXXI', 'IIIIXX', 'XIIIIX'], ['YYIIII', 'IYYIII', 'IIYYII', 'IIIYYI', 'IIIIYY', 'YIIIIY'], ['ZZIIII', 'IZZIII', 'IIZZII', 'IIIZZI', 'IIIIZZ', 'ZIIIIZ'], ['ZIIIII'], ['IZIIII'], ['IIZIII'], ['IIIZII'], ['IIIIZI'], ['IIIIIZ']]


In [ ]:
t = 1
M = 15

rho = rand_rho(n)
Ns = [2**i + 10 for i in range(5, M)]
st = time.time()
errors_costs1 = np.array([Error_cost(Hm, Hs, hs, t, rho, N, M=100, threads=12) for N in Ns])
print(time.time()-st)
errors_costs = np.array([Error_cost(Hm, Hs_s, hs_s, t, rho, N, icosts, M=100, threads=12) for N in Ns])
errors, errors1 = errors_costs[:, 0], errors_costs1[:, 0]
tcosts, rcosts = errors_costs[:, 1], errors_costs[:, 2]

Running N=74:  21%|██        | 21/100 [00:06<00:21,  3.61it/s]

In [ ]:
plt.title('Iterations vs Error')
plt.plot(errors, Ns, 'b-', label='Multiple terms', markersize=3)
plt.plot(errors1, Ns, 'r-', label='Single term', markersize=3)
plt.legend()
plt.show()

In [ ]:
plt.title('Total cost vs Error')
plt.plot(errors, tcosts, 'b-', label='Multiple terms', markersize=3)
plt.plot(errors1, Ns, 'r-', label='Single term', markersize=3)
plt.legend()
plt.show()

In [ ]:
plt.title('Rotations vs Error')
plt.plot(errors, rcosts, 'b-', label='Multiple terms', markersize=3)
plt.plot(errors1, Ns, 'r-', label='Single term', markersize=3)
plt.legend()
plt.show()

In [ ]:
plt.title('log Iterations vs log Error')
plt.plot(np.log10(errors), np.log10(Ns), 'b-', label='Multiple terms', markersize=3)
plt.plot(np.log10(errors1), np.log10(Ns), 'r-', label='Single term', markersize=3)
plt.legend()
plt.show()

In [ ]:
toffcost = tcosts - rcosts

plt.title('log Toffoli cost vs log Error')
plt.plot(np.log10(errors), np.log10(toffcost), 'b-', label='Multiple terms', markersize=3)
plt.legend()
plt.show()

In [ ]:
plt.title('log Rotations vs log Error')
plt.plot(np.log10(errors), np.log10(rcosts), 'b-', label='Multiple terms', markersize=3)
plt.plot(np.log10(errors1), np.log10(Ns), 'r-', label='Single term', markersize=3)
plt.legend()
plt.show()

In [ ]:
print(errors1[-1], errors[-1])

In [ ]:
np.savetxt('save/rcosts-6H1d2.txt', rcosts)
np.savetxt('save/tcosts-6H1d2.txt', tcosts)
np.savetxt('save/errors-6H1d2.txt', errors)
np.savetxt('save/errors1-6H1d2.txt', errors1)
np.savetxt('save/Ns-6H1d2.txt', Ns) 

In [ ]:
def cost_for_error(H, Hs, hs, t, eps, icosts=None, step=100):
    """ Binary search for first N where N segments has error <= eps
    """
    error = 1e5
    N = int(np.ceil(2*np.sum(np.abs(hs))**2*t**2/eps))
    U = scipy.linalg.expm(1j*t*H)
    while error > eps:
        Vlist, lamb, _ = QDrift(hs, t, eps, N)
        tau = t*lamb/N
        Vs = [scipy.linalg.expm(1j*tau*Hs[i]) for i in Vlist]
        V = reduce(np.matmul, Vs)
        error = np.linalg.norm(V-U)
        cost = (np.sum([icosts[Vlist[i]] for i in range(N)]) 
                if icosts is not None else N)
        print('N={0}, error={1}'.format(N, error))
        N *= 2
    
    low, high, mid, last = N//2, N, N//2, N
    Vlist, lamb, _ = QDrift(hs, t, eps, high)
    while low <= high:
        mid =  (high + low)//2
        tau = t*lamb/mid
        Vs = [scipy.linalg.expm(1j*tau*Hs[i]) for i in Vlist][0:mid]
        V = reduce(np.matmul, Vs)
        error = np.linalg.norm(V-U)
        cost = (np.sum([icosts[Vlist[i]] for i in range(mid)]) 
                if icosts is not None else mid)
        print('last={0}, error={1}'.format(last, error))
        if error > eps:
            low = mid+1
        elif error < eps:
            high, last = mid-1, mid
        else:
            high, last = mid, mid
    cost = (np.sum([icosts[Vlist[i]][0] + icosts[Vlist[i]][1] for i in range(last)]) 
                if icosts is not None else last)
    return cost